## Demo for Pytorch implementation of baseline

In [1]:
import sys

# Add main code-base to path
sys.path.append('../')

from util_rename import *
from model import ImgSegRefExpModel
import torch
import sys
import skimage.io
import numpy as np
import tensorflow as tf
import json
import timeit
from utl import eval_tools, im_processing
#import data
from data import ImageSegmentationDataset
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
# trained model
pretrained_model_file = "/Users/shubhammehrotra/Downloads/text_objseg_pretrained_torch_converted_with_lstm"
vocab_file = './vocabulary_referit.txt'

root = '/Users/shubhammehrotra/text_objseg/exp-referit/'

image_dir = root + 'referit-dataset/images/'
mask_dir = root + 'referit-dataset/mask/'
query_file = root + 'data/referit_query_test.json'
bbox_file = root + 'data/referit_bbox.json'
imcrop_file = root + 'data/referit_imcrop.json'
imsize_file = root + 'data/referit_imsize.json'
vocab_file = root + 'data/vocabulary_referit.txt'

In [3]:
# Load vocabulary
vocab_dict = load_vocab_dict_from_file(vocab_file)

# Load model and weights
model = ImgSegRefExpModel(mlp_hidden=500, vocab_size=8803, emb_size=1000, lstm_hidden_size=1000)
pre_trained = torch.load(pretrained_model_file)
model.load_state_dict(pre_trained)

In [4]:
set(model.state_dict().keys()).difference(set(pre_trained.keys()))

set()

In [5]:
test_dataset = ImageSegmentationDataset(query_file, image_dir, mask_dir)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)
sum_I = 0
sum_U = 0

In [12]:
for batch_idx, (image, text, mask, original_image) in enumerate(test_loader):
    print ('Image Number:', batch_idx)
    text = text.long()
    output_mask = model((image, text))
    output_mask = model((image, text))
    
    labels = mask > 0
    #labels = mask.cpu().detach().numpy() > 0
    #print("Mask shape", output_mask.shape)
    output_mask = output_mask.squeeze(0)
    output_mask = output_mask.squeeze(0)
    #print("Mask shape squeezed", output_mask.shape)
    
    image_name = ""
    for digit in original_image[0]:
        #print (digit.item())
        image_name += str(digit.item())

    #print (int (q))
    im = skimage.io.imread('/Users/shubhammehrotra/text_objseg/exp-referit/referit-dataset/images/' + image_name + '.jpg')
    
    #prediction = resize_and_crop(output_mask > 0, * im.shape[:2]).astype(np.bool)
    # Final prediction
    prediction = resize_and_crop(output_mask.cpu().detach().numpy() > 0, * im.shape[:2]).astype(np.bool)
    #print (prediction)
    prediction = im_processing.resize_and_crop(prediction, im.shape[0], im.shape[1])
    I, U = eval_tools.compute_mask_IU(prediction, labels)
    I = float(I)
    U = float(U)
    sum_I += I
    sum_U += U
    print ('Image IOU', I / U)
    print ('Updated IOU', sum_I / sum_U)
    print ('\n')
    break

print ('Final IOU', float(sum_I)/sum_U)


Image Name /Users/shubhammehrotra/text_objseg/exp-referit/referit-dataset/images/30456
Image Text sky
Image Number: 0
torch.Size([1, 1000, 16, 16])
16 16
torch.Size([1, 1000, 16, 16])
16 16


TypeError: sum() received an invalid combination of arguments - got (out=NoneType, axis=NoneType, ), but expected one of:
 * ()
 * (torch.dtype dtype)
 * (tuple of ints dim, torch.dtype dtype)
      didn't match because some of the keywords were incorrect: out, axis
 * (tuple of ints dim, bool keepdim, torch.dtype dtype)
 * (tuple of ints dim, bool keepdim)
      didn't match because some of the keywords were incorrect: out, axis


In [ ]:
#Displaying output
   
im = skimage.io.imread('/Users/shubhammehrotra/text_objseg/exp-referit/referit-dataset/images/' + q + '.jpg')

    # Final prediction
prediction = resize_and_crop(output_mask.cpu().detach().numpy() > 0, * im.shape[:2]).astype(np.bool)
#print (prediction)
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(im)
plt.subplot(1, 2, 2)
plt.imshow(prediction)
#print (labels.shape, prediction.shape)
#print ()
prediction = im_processing.resize_and_crop(prediction, im.shape[0], im.shape[1])
I, U = eval_tools.compute_mask_IU(prediction, labels)

print ('IOU',  I/U)


#         # Evaluate the segmentation performance of using bounding box segmentation
#         pred_raw = (scores_val >= score_thresh).astype(np.float32)
#         predicts = im_processing.resize_and_crop(pred_raw, im.shape[0], im.shape[1])
#         I, U = eval_tools.compute_mask_IU(predicts, labels)
#         cum_I += I
#         cum_U += U
#         this_IoU = I/U

#print("query text = '%s'" % query)



In [ ]:
original_image_name = "12346"
char = []
    
char = [int(i) for i in original_image_name]

char = torch.from_numpy(np.array(char))

q = ""
for digit in s:
    print (digit)
    q += str(digit)

print (int (q))



### Run on sample image

In [ ]:
# input image and query text
im_file = './38100.jpg'
query = 'sky above the bridge'

In [ ]:
# Load the model
# Model Param
T = 20
N = 1
input_H = 512; featmap_H = (input_H // 32)
input_W = 512; featmap_W = (input_W // 32)
num_vocab = 8803
embed_dim = 1000
lstm_dim = 1000
mlp_hidden_dims = 500

channel_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32)

# Run on the input image and query text
text_seq_val = np.zeros((T, N), dtype=np.float32)
imcrop_val = np.zeros((N, input_H, input_W, 3), dtype=np.float32)

# Preprocess image and text
im = skimage.io.imread(im_file)

# Makes it uint8
processed_im = skimage.img_as_ubyte(resize_and_pad(im, input_H, input_W))
print(processed_im.dtype)

imcrop_val[0, :] = processed_im.astype(np.float32) - channel_mean
print(imcrop_val.dtype)

# preprocess sentence pads 0's before the phrase. flipping it ruins the output
text_seq_val[:, 0] = preprocess_sentence(query, vocab_dict, T)
print(text_seq_val.dtype)
print (text_seq_val)

imcrop_val = torch.Tensor(imcrop_val).permute(0, 3, 1, 2) 
text_seq_val = torch.LongTensor(text_seq_val).t() 

In [ ]:
print(imcrop_val.shape)
print(text_seq_val.shape)

In [ ]:
# Forward pass to get response map
output_mask = model((imcrop_val, text_seq_val))

print("Mask shape", output_mask.shape)
output_mask = output_mask.squeeze(0)
output_mask = output_mask.squeeze(0)
print("Mask shape squeezed", output_mask.shape)

# Final prediction
prediction = resize_and_crop(output_mask.cpu().detach().numpy() > 0, * im.shape[:2]).astype(np.bool)

In [ ]:
prediction

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(im)
plt.subplot(1, 2, 2)
plt.imshow(prediction)

print("query text = '%s'" % query)